In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import torchvision
from torchvision import models, transforms, datasets
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
import torchattacks
from torchattacks import OnePixel, Pixle, DeepFool, CW

In [2]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = './'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_ft = models.efficientnet_b0(pretrained=True)
num_ftrs = model_ft.classifier[1].in_features
model_ft.fc = nn.Linear(num_ftrs, 2)
model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

model_weights_path = "models/efficientNetB0_binary_ucf_crime.pth"

model_ft.load_state_dict(torch.load(model_weights_path, map_location=torch.device('cpu')), strict=False)

test_loader = dataloaders["test"]

model = model_ft

/home/qaiser/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/qaiser/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
images, labels = next(iter(test_loader))

In [4]:
import torch
import torchvision.transforms as transforms
import torchvision.utils as vutils

def save_rgb(adv_images, filename):
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    
    denormalize = transforms.Normalize(
        mean=[-m/s for m, s in zip(mean, std)],
        std=[1/s for s in std]
    )
    
    adv_images_denorm = adv_images.clone()
    
    for i in range(adv_images_denorm.size(0)):
        adv_images_denorm[i] = denormalize(adv_images_denorm[i])
    
    adv_images_denorm = torch.clamp(adv_images_denorm, 0, 1)

    ### new code start ###
    for i, adv_image in enumerate(adv_images_denorm):
        # Convert the tensor to an image format
        adv_image_rgb = transforms.ToPILImage()(adv_image.cpu())
        
        # Save the image
        adv_image_rgb.save(f"adversarial_image_{i}.png")
    ### new code end ###
    
    vutils.save_image(adv_images_denorm, filename)

In [5]:
pixle = torchattacks.Pixle(model,
        x_dimensions=(1, 2),
        y_dimensions=(1, 2),
        pixel_mapping="random",
        restarts=5,
        max_iterations=10,
        update_each_iteration=False,)

onepixel = torchattacks.OnePixel(
    model, 
    pixels=5,        # Start with a single pixel for minimal perturbation
    steps=30,        # Increase the steps for better optimization
    popsize=34,      # Increase population size for better candidate exploration
    inf_batch=4    # Standard batch size, can be increased if memory allows
)

In [7]:
mpd = torchattacks.MultiAttack([pixle, onepixel])

### One Pixel Generated Samples

In [8]:
adv_images = onepixel(images, labels)

In [9]:
save_rgb(adv_images, "RGB_onepixel_adversarial_samples.png")

### Pixle Generated Samples

In [10]:
adv_images_pixle = pixle(images, labels)

In [11]:
save_rgb(adv_images_pixle, "RGB_pixle_adversarial_samples.png")

### MPD Generated Samples

In [12]:
adv_images_mpd = mpd(images, labels)

In [13]:
save_rgb(adv_images_mpd, "RGB_mpd_adversarial_samples.png")